In [1]:
import pickle
import random
import random
import spacy
import csv
import sys
import errno
import glob
import string
import io
import os
import re
import time
import functools
import numpy as np
import pandas as pd
from setuptools import setup
from collections import Counter
from collections import defaultdict
from argparse import ArgumentParser

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset
import torch.nn.functional as F
from torch.autograd import Variable

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
PAD_IDX = 0
UNK_IDX = 1
label_dict = {"entailment":0, "neutral":1, "contradiction":2}

In [3]:
no_cuda = False
cuda = not no_cuda and torch.cuda.is_available()
seed = 1
device = torch.device("cuda" if cuda else "cpu")

In [4]:
opus_path = "/scratch/adc563/nlu_project/data/opus"
europarl_path = "/scratch/adc563/nlu_project/data/europarl"
un_path = "/scratch/adc563/nlu_project/data/un_parallel_corpora"

In [5]:
def read_xnli(lang):
    fname = "/scratch/adc563/nlu_project/data/XNLI/xnli.{}.jsonl"
    xnli_dev = pd.read_json(fname.format("dev"), lines=True)
    xnli_test = pd.read_json(fname.format("test"), lines=True)
    if lang == "all":
        dev_data = xnli_dev
        test_data = xnli_test
    else:
        dev_data = xnli_dev[xnli_dev["language"]==lang]
        test_data = xnli_test[xnli_test["language"]==lang]
    return dev_data, test_data

def load_aligned_vectors(lang):
    f = "/scratch/adc563/nlu_project/data/aligned_embeddings/wiki.{}.align.vec".format(lang)
    fin = io.open(f, "r", encoding="utf-8", newline="\n", errors="ignore")
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(" ")
        data[tokens[0]] = [*map(float, tokens[1:])]
    return data

def load_multilingual_vectors(lang):
    f = "/scratch/adc563/nlu_project/data/multi_lingual_embeddings/cc.{}.300.vec".format(lang)
    fin = io.open(f, "r", encoding="utf-8", newline="\n", errors="ignore")
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(" ")
        data[tokens[0]] = [*map(float, tokens[1:])]
    return data

def load_glove_vectors(lang):
    f = "/scratch/adc563/nlu_project/HBMP/vector_cache/glove.840B.300d.txt".format(lang)
    fin = io.open(f, "r", encoding="utf-8", newline="\n", errors="ignore")
    n = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(" ")
        data[tokens[0]] = [*map(float, tokens[1:])]
    return data

def read_enli(nli_corpus = "snli"):
    if nli_corpus == "snli":
        path_ = "/scratch/adc563/nlu_project/HBMP/data/snli/snli_1.0/snli_1.0"
        train = pd.read_json("{}_{}.jsonl".format(path_,"train"), lines=True)
        dev = pd.read_json("{}_{}.jsonl".format(path_,"dev"), lines=True)
        test = pd.read_json("{}_{}.jsonl".format(path_,"test"), lines=True)
        # remove - from gold label
        train = train[train["gold_label"] != "-"]
        dev = dev[dev["gold_label"] != "-"]
        test = test[test["gold_label"] != "-"]
    elif nli_corpus == "multinli":
        path_ = "/scratch/adc563/nlu_project/HBMP/data/multinli/multinli_1.0/multinli_1.0"
        train = pd.read_json("{}_{}.jsonl".format(path_,"train"), lines=True)
        dev = pd.read_json("{}_{}_matched.jsonl".format(path_, "dev"), lines=True)
        test = None
        # remove - from gold label
        train = train[train["gold_label"] != "-"]
        dev = dev[dev["gold_label"] != "-"]
    return train, dev, test

def write_numeric_label(train, dev, test, nli_corpus="multinli"):
    if nli_corpus == "multinli":
        for dataset in [train, dev]:
            dataset["gold_label"] = dataset["gold_label"].apply(lambda x: label_dict[x])
    elif nli_corpus == "snli":
        for dataset in [train, dev, test]:
            dataset["gold_label"] = dataset["gold_label"].apply(lambda x: label_dict[x])
    elif nli_corpus == "xnli":
        for dataset in [dev, test]:
            dataset["gold_label"] = dataset["gold_label"].apply(lambda x: label_dict[x])
    else:
        raise ValueError ("NLI corpus name should be in [multinli, snli, xnli]")
    return train, dev, test

def tokenize_xnli(dataset, remove_punc=False, lang="en"):
    all_s1_tokens = []
    all_s2_tokens = []
    for s in ["sentence1", "sentence2"]:
        punc = [*string.punctuation]
        dataset["{}_tokenized".format(s)] = dataset["{}".format(s)].\
        apply(lambda x: "".join(c for c in x if c not in string.punctuation).lower().split(" "))
        dataset["{}_tokenized".format(s)] = dataset["{}_tokenized".format(s)].\
        apply(lambda x: [a+"."+lang for a in x])
    ext = dataset["sentence1_tokenized"].apply(lambda x: all_s1_tokens.extend(x))
    ext1 = dataset["sentence2_tokenized"].apply(lambda x: all_s2_tokens.extend(x))
    all_tokens = all_s1_tokens + all_s2_tokens
    return dataset, all_tokens

def build_vocab(all_tokens, max_vocab_size):
    token_counter = Counter(all_tokens)
    vocab, count = zip(*token_counter.most_common(max_vocab_size))
    id2token = [*vocab]
    token2id = dict(zip(vocab, range(2,2+len(vocab))))
    id2token = ['<PAD>', '<UNK>'] + id2token
    token2id["<PAD>"] = 0
    token2id["<UNK>"] = 1
    return token2id, id2token



def build_tok2id(id2token):
    token2id = {}
    for i in range(len(id2token)):
        token2id[id2token[i]] = i
    return token2id

def init_embedding_weights(vectors, token2id, id2token, embedding_size):
    weights = np.zeros((len(id2token), embedding_size))
    for idx in range(2, len(id2token)):
        token = id2token[idx]
        weights[idx] = vectors[token]
    weights[1] = np.random.randn(embedding_size)
    return weights

In [6]:
class NLIconfig:
    def __init__(self, corpus, val_test_lang, max_sent_len, max_vocab_size, epochs, batch_size, 
                    embed_dim, hidden_dim, dropout, lr, experiment_lang):
        self.corpus = corpus
        self.val_test_lang = val_test_lang
        self.max_sent_len = max_sent_len
        self.max_vocab_size = max_vocab_size
        self.epochs = epochs
        self.batch_size = batch_size
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.dropout = dropout
        self.lr = lr
        self.experiment_lang = experiment_lang

In [7]:
config = NLIconfig(corpus = "multinli",
             val_test_lang = "en",
             max_sent_len = 40,
             max_vocab_size = 200000,
             epochs = 15,
             batch_size = 32,
             embed_dim = 300,
             hidden_dim = 512,
             dropout = 0.09,
             lr = 1e-3,
             experiment_lang = "ru")

In [8]:
def update_vocab_keys(src_vocab, trg_vocab):
    for x in [*src_vocab.keys()]:
        src_vocab[x + ".en"] = src_vocab[x]
        src_vocab.pop(x)
    for y in [*trg_vocab.keys()]:
        trg_vocab[y + ".{}".format(config.experiment_lang)] = trg_vocab[y]
        trg_vocab.pop(y)
        
    src_vocab.update(trg_vocab)
    return src_vocab

In [9]:
print ("Loading vectors for EN.")
aligned_src_vectors = load_glove_vectors("en")
print ("Loading vectors for {}.".format(config.experiment_lang.upper()))
aligned_trg_vectors = load_aligned_vectors(config.experiment_lang)

Loading vectors for EN.
Loading vectors for RU.


In [10]:
id2token_src = [x+"."+"en" for x in [*aligned_src_vectors.keys()]][:config.max_vocab_size]
id2token_trg = [x+".{}".format(config.experiment_lang) for x in [*aligned_trg_vectors.keys()]][:config.max_vocab_size]
id2token_mutual = ["<PAD>", "<UNK>"] + id2token_src + id2token_trg
vecs_mutual = update_vocab_keys(aligned_src_vectors, aligned_trg_vectors)

In [11]:
token2id_mutual = build_tok2id(id2token_mutual)
weights_init = init_embedding_weights(vecs_mutual, token2id_mutual, id2token_mutual, 300)

In [12]:
# load train and preprocess
print ("Reading {} data.".format(config.corpus))
nli_train, nli_dev, nli_test = read_enli(nli_corpus=config.corpus)
print ("Writing numeric label.")
if config.corpus == "multinli":
    nli_train, nli_dev, _ = write_numeric_label(nli_train, nli_dev, nli_test, nli_corpus=config.corpus)
elif config.corpus == "snli":
    nli_train, nli_dev, nli_test = write_numeric_label(nli_train, nli_dev, nli_test, nli_corpus=config.corpus)

# load val and test and preprocess
print ("Reading XNLI {} data.".format(config.val_test_lang.upper()))
xnli_dev, xnli_test = read_xnli(config.val_test_lang)
_, xnli_dev, xnli_test = write_numeric_label(None, xnli_dev, xnli_test, nli_corpus="xnli")

Reading multinli data.
Writing numeric label.
Reading XNLI EN data.


In [13]:
nli_train, all_train_tokens = tokenize_xnli(nli_train, lang="en")
# nli_dev = tokenize_enli(nli_dev)
xnli_dev, _ = tokenize_xnli(xnli_dev, lang="en")
xnli_test, _ = tokenize_xnli(xnli_test, lang="en")

In [14]:
nli_train.head(3)

,annotator_labels,genre,gold_label,pairID,promptID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse,sentence1_tokenized,sentence2_tokenized
0,[neutral],government,1,31193n,31193,Conceptually cream skimming has two basic dime...,( ( Conceptually ( cream skimming ) ) ( ( has ...,(ROOT (S (NP (JJ Conceptually) (NN cream) (NN ...,Product and geography are what make cream skim...,( ( ( Product and ) geography ) ( ( are ( what...,(ROOT (S (NP (NN Product) (CC and) (NN geograp...,"[conceptually.en, cream.en, skimming.en, has.e...","[product.en, and.en, geography.en, are.en, wha..."
1,[entailment],telephone,0,101457e,101457,you know during the season and i guess at at y...,( you ( ( know ( during ( ( ( the season ) and...,(ROOT (S (NP (PRP you)) (VP (VBP know) (PP (IN...,You lose the things to the following level if ...,( You ( ( ( ( lose ( the things ) ) ( to ( the...,(ROOT (S (NP (PRP You)) (VP (VBP lose) (NP (DT...,"[you.en, know.en, during.en, the.en, season.en...","[you.en, lose.en, the.en, things.en, to.en, th..."
2,[entailment],fiction,0,134793e,134793,One of our number will carry out your instruct...,( ( One ( of ( our number ) ) ) ( ( will ( ( (...,(ROOT (S (NP (NP (CD One)) (PP (IN of) (NP (PR...,A member of my team will execute your orders w...,( ( ( A member ) ( of ( my team ) ) ) ( ( will...,(ROOT (S (NP (NP (DT A) (NN member)) (PP (IN o...,"[one.en, of.en, our.en, number.en, will.en, ca...","[a.en, member.en, of.en, my.en, team.en, will...."


In [15]:
class NLIDataset(Dataset):
    def __init__(self, tokenized_dataset, max_sentence_length, token2id, id2token):
        self.sentence1, self.sentence2, self.labels = [*tokenized_dataset["sentence1_tokenized"].values], \
                                                      [*tokenized_dataset["sentence2_tokenized"].values], \
                                                      [*tokenized_dataset["gold_label"].values]
        self.max_sentence_length = int(max_sentence_length)
        self.token2id, self.id2token = token2id, id2token
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, row):
        label = self.labels[row]
        sentence1_word_idx, sentence2_word_idx = [], []
        sentence1_mask, sentence2_mask = [], []
        for word in self.sentence1[row][:self.max_sentence_length]:
            if word in self.token2id.keys():
                sentence1_word_idx.append(self.token2id[word])
                sentence1_mask.append(0)
            else:
                sentence1_word_idx.append(UNK_IDX)
                sentence1_mask.append(1)
        for word in self.sentence2[row][:self.max_sentence_length]:
            if word in self.token2id.keys():
                sentence2_word_idx.append(self.token2id[word])
                sentence2_mask.append(0)
            else:
                sentence2_word_idx.append(UNK_IDX)
                sentence2_mask.append(1)
        sentence1_list = [sentence1_word_idx, sentence1_mask, len(sentence1_word_idx)]
        sentence2_list = [sentence2_word_idx, sentence2_mask, len(sentence2_word_idx)]
        
        return sentence1_list + sentence2_list + [label]

def nli_collate_func(batch, max_sent_length):
    sentence1_data, sentence2_data = [], []
    sentence1_mask, sentence2_mask = [], []
    s1_lengths, s2_lengths = [], []
    labels = []

    for datum in batch:
        s1_lengths.append(datum[2])
        s2_lengths.append(datum[5])
        labels.append(datum[6])
        sentence1_data_padded = np.pad(np.array(datum[0]), pad_width=((0, config.max_sent_len-datum[2])), mode="constant", constant_values=0)
        sentence1_data.append(sentence1_data_padded)
        sentence1_mask_padded = np.pad(np.array(datum[1]), pad_width=((0, config.max_sent_len-datum[2])), mode="constant", constant_values=0)
        sentence1_mask.append(sentence1_mask_padded)
        sentence2_data_padded = np.pad(np.array(datum[3]), pad_width=((0, config.max_sent_len-datum[5])), mode="constant", constant_values=0)
        sentence2_data.append(sentence2_data_padded)
        sentence2_mask_padded = np.pad(np.array(datum[4]), pad_width=((0, config.max_sent_len-datum[5])), mode="constant", constant_values=0)
        sentence2_mask.append(sentence2_mask_padded)
        
    ind_dec_order = np.argsort(s1_lengths)[::-1]
    sentence1_data = np.array(sentence1_data)[ind_dec_order]
    sentence2_data = np.array(sentence2_data)[ind_dec_order]
    sentence1_mask = np.array(sentence1_mask)[ind_dec_order].reshape(len(batch), -1, 1)
    sentence2_mask = np.array(sentence2_mask)[ind_dec_order].reshape(len(batch), -1, 1)
    s1_lengths = np.array(s1_lengths)[ind_dec_order]
    s2_lengths = np.array(s2_lengths)[ind_dec_order]
    labels = np.array(labels)[ind_dec_order]
    
    s1_list = [torch.from_numpy(sentence1_data), torch.from_numpy(sentence1_mask).float(), s1_lengths]
    s2_list = [torch.from_numpy(sentence2_data), torch.from_numpy(sentence2_mask).float(), s2_lengths]
        
    return [torch.from_numpy(sentence1_data), torch.from_numpy(sentence1_mask).float(), s1_lengths,
            torch.from_numpy(sentence2_data), torch.from_numpy(sentence2_mask).float(), s2_lengths,
            labels]

In [16]:
# train
nli_train_dataset = NLIDataset(nli_train, max_sentence_length=config.max_sent_len, token2id=token2id_mutual, id2token=id2token_mutual)
nli_train_loader = torch.utils.data.DataLoader(dataset=nli_train_dataset, batch_size=config.batch_size,
                               collate_fn=lambda x, max_sentence_length=config.max_sent_len: nli_collate_func(x, config.max_sent_len),
                               shuffle=False)

# dev
nli_dev_dataset = NLIDataset(xnli_dev, max_sentence_length=config.max_sent_len, token2id=token2id_mutual, id2token=id2token_mutual)
nli_dev_loader = torch.utils.data.DataLoader(dataset=nli_dev_dataset, batch_size=config.batch_size,
                               collate_fn=lambda x, max_sentence_length=config.max_sent_len: nli_collate_func(x, config.max_sent_len),
                               shuffle=False)

# test
nli_test_dataset = NLIDataset(xnli_test, max_sentence_length=config.max_sent_len, token2id=token2id_mutual, id2token=id2token_mutual)
nli_test_loader = torch.utils.data.DataLoader(dataset=nli_test_dataset, batch_size=config.batch_size,
                               collate_fn=lambda x, max_sentence_length=config.max_sent_len: nli_collate_func(x, config.max_sent_len),
                               shuffle=False)

In [17]:
class biLSTM(nn.Module):
    
    def __init__(self,
                 hidden_size,
                 embedding_weights,
                 percent_dropout,
                 vocab_size,
                 interaction_type="concat",
                 num_layers=1,
                 input_size=300,
                 src_trg = "src"):

        super(biLSTM, self).__init__()
        
        self.num_layers, self.hidden_size = num_layers, hidden_size
        
        self.embed_table = torch.from_numpy(embedding_weights).float()
        embedding = nn.Embedding.from_pretrained(self.embed_table)

        self.embedding = embedding
        self.interaction = interaction_type
        self.dropout = percent_dropout
        self.drop_out = nn.Dropout(self.dropout)
        
        self.LSTM = nn.LSTM(300, hidden_size, num_layers, batch_first=True, bidirectional=True)
#         self.LSTM2 = nn.LSTM(300, hidden_size, num_layers, batch_first=True, bidirectional=True)
#         self.LSTM3 = nn.LSTM(300, hidden_size, num_layers, batch_first=True, bidirectional=True)
        
        if self.LSTM.bidirectional:
            self.num_directions = 2
        else:
            self.num_directions = 1
            
        self.bn = nn.BatchNorm1d(self.hidden_size * self.num_directions)
        
    def init_hidden(self, batch_size):
        hidden = torch.randn(self.num_directions*self.num_layers, batch_size, self.hidden_size).to(device)
        c_0 = torch.randn(self.num_directions*self.num_layers, batch_size, self.hidden_size).to(device)
        return hidden, c_0
    
    def forward(self, sentence, mask, lengths):
        sort_original = sorted(range(len(lengths)), key=lambda sentence: -lengths[sentence])
        unsort_to_original = sorted(range(len(lengths)), key=lambda sentence: sort_original[sentence])
        
        sentence = sentence[sort_original]
        _mask = mask[sort_original]
        lengths = lengths[sort_original]
        batch_size, seq_len = sentence.size()
        self.hidden, self.c_0 = self.init_hidden(batch_size)
        
        # embdddings
        embeds = self.embedding(sentence)
        embeds = mask*embeds + (1-_mask)*embeds.clone().detach()
        embeds = torch.nn.utils.rnn.pack_padded_sequence(embeds, lengths, batch_first=True)
        # first lstm
        lstm_out, (self.hidden_1, self.c_1) = self.LSTM(embeds, (self.hidden, self.c_0))
        emb1, _ = torch.nn.utils.rnn.pad_packed_sequence(lstm_out, batch_first=True)
        
        emb1 = emb1.view(batch_size, -1, 2, self.hidden_size)
        emb1 = torch.max(emb1, dim=1)[0]
        emb1 = torch.cat([emb1[:,i,:] for i in range(self.num_directions)], dim=1)
        emb1 = emb1[unsort_to_original]
        
        out = self.bn(emb1)
        
#         lstm_out_2, (self.hidden_2, self.c_2) = self.LSTM2(embeds, (self.hidden_1, self.c_1))
#         lstm_out_2, _ = torch.nn.utils.rnn.pad_packed_sequence(lstm_out_2, batch_first=True)
#         lstm_out_2 = lstm_out_2.view(batch_size, -1, 2, self.hidden_size)
        
#         lstm_out_2 = torch.max(lstm_out_2, dim=1)[0]
#         lstm_out_2 = torch.cat([lstm_out_2[:,i,:] for i in range(self.num_directions)], dim=1)
#         lstm_out_2 = lstm_out_2[unsort_to_original]
        
#         lstm_out_3, (self.hidden_3, self.c_3) = self.LSTM3(embeds, (self.hidden_2, self.c_2))
#         lstm_out_3, _ = torch.nn.utils.rnn.pad_packed_sequence(lstm_out_3, batch_first=True)
#         lstm_out_3 = lstm_out_3.view(batch_size, -1, 2, self.hidden_size)
#         lstm_out_3 = torch.max(lstm_out_3, dim=1)[0]
        
#         lstm_out_3 = torch.cat([lstm_out_3[:,i,:] for i in range(self.num_directions)], dim=1)
#         lstm_out_3 = lstm_out_3[unsort_to_original]
#         out = torch.cat([emb1, lstm_out_2, lstm_out_3], dim=1)
        
        return out

In [18]:
class Linear_Layers(nn.Module):
    
    def __init__(self, hidden_size, hidden_size_2, percent_dropout,
                 interaction_type="concat", classes=3, input_size=300):
        
        super(Linear_Layers, self).__init__()
        self.interaction = interaction_type
        self.num_classes = classes
        self.hidden_size = hidden_size
        self.hidden_size_2 = hidden_size_2
        self.percent_dropout = percent_dropout
        self.num_classes = classes
        
        if self.interaction == "concat":
            self.mlp = nn.Sequential(
                nn.Linear(4 * self.hidden_size, self.hidden_size_2),
                nn.ReLU(inplace=True),
                nn.Dropout(p=self.percent_dropout),
                nn.Linear(self.hidden_size_2, self.num_classes))
        else:
            self.mlp = nn.Sequential(
                nn.Dropout(p=self.percent_dropout),
                nn.Linear(6 * self.hidden_size, self.hidden_size_2),
                nn.ReLU(inplace=True),
                nn.Dropout(p=self.percent_dropout),
                nn.Linear(self.hidden_size_2, int(self.hidden_size_2/2)),
                nn.ReLU(inplace=True),
                nn.Linear(int(self.hidden_size_2/2), self.num_classes))

        self.init_weights()
        
    def init_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_normal_(module.weight)
                nn.init.uniform_(module.bias)

    def forward(self, lstm_out_1, lstm_out_2):
        if self.interaction == "concat":
            hidden = torch.cat([lstm_out_1, lstm_out_2, torch.abs(lstm_out_1 - lstm_out_2), 
                                torch.mul(lstm_out_1, lstm_out_2)], dim=1)
        elif self.interaction == "mul":
            hidden = lstm_out_1*lstm_out_2
        elif self.interaction == "subtract":
            hidden = lstm_out_1-lstm_out_2
        hidden = hidden.view(hidden.size(0),-1) 
        out = self.mlp(hidden)
        out = F.log_softmax(out, 1)
        return out

In [19]:
def train(RNN, Linear_Classifier, DataLoader, criterion, optimizer, epoch):
    
    RNN.train()
    Linear_Classifier.train()
    total_loss = 0
    for batch_idx, (sentence1, s1_original, sentence1_lengths, 
                    sentence2, s2_original, sentence2_lengths, labels)\
    in enumerate(DataLoader):
        sentence1, s1_original = sentence1.to(device), s1_original.to(device),  
        sentence2, s2_original = sentence2.to(device), s2_original.to(device),
        labels = torch.from_numpy(labels).to(device)
        RNN.train()
        Linear_Classifier.train()
        optimizer.zero_grad()
        output_s1 = RNN(sentence1, s1_original, sentence1_lengths)
        output_s2 = RNN(sentence2, s2_original, sentence2_lengths)
        out = Linear_Classifier(output_s1, output_s2)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * len(sentence1) / len(DataLoader.dataset)
        if (batch_idx+1) % (len(DataLoader.dataset)//(20*labels.shape[0])) == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_idx+1) * labels.shape[0], len(DataLoader.dataset),
                100. * (batch_idx+1) / len(DataLoader), loss.item()), end="\r")

    optimizer.zero_grad()
    return total_loss


def test(RNN, Linear_Classifier, DataLoader, criterion):

    RNN.eval()
    Linear_Classifier.eval()
    test_loss = 0
    label_list = []
    output_list = []
    with torch.no_grad():
        for batch_idx, (sentence1, s1_original, sentence1_lengths, 
                    sentence2, s2_original, sentence2_lengths, labels)\
                    in enumerate(DataLoader):
            sentence1, s1_original = sentence1.to(device), s1_original.to(device),  
            sentence2, s2_original = sentence2.to(device), s2_original.to(device),
            labels = torch.from_numpy(labels).to(device)
            output_s1 = RNN(sentence1, s1_original, sentence1_lengths)
            output_s2 = RNN(sentence2, s2_original, sentence2_lengths)
            out = Linear_Classifier(output_s1, output_s2)
            loss = criterion(out, labels)
            test_loss += loss.item()/len(DataLoader.dataset)
            output_list.append(out)
            label_list.append(labels)
            
    return test_loss, torch.cat(output_list, dim=0), torch.cat(label_list, dim=0)

def accuracy(RNN, Linear_Classifier, DataLoader, criterion):
    
    _, predicted, true_labels = test(RNN = RNN,  Linear_Classifier = Linear_Classifier,
                                     DataLoader = DataLoader, criterion = criterion)

    predicted = predicted.max(1)[1]
    return 100 * predicted.eq(true_labels.data.view_as(predicted)).float().mean().item()

In [20]:
n_epochs = 20

In [21]:
LSTM_en = biLSTM(config.hidden_dim, weights_init, config.dropout, config.max_vocab_size, 
                 interaction_type="concat",
                 num_layers=1, input_size=300, src_trg = "src").to(device)

LSTM_en.embedding.requires_grad = False

linear_model = Linear_Layers(hidden_size = 1024, hidden_size_2 = 128,
                             percent_dropout = 0.1, interaction_type="concat", 
                             classes=3, input_size=300).to(device)

print ("Encoder:\n", LSTM_en)
print ("Classifier:\n", linear_model)
validation_accuracy = [0]
start_epoch = 0
for epoch in range(start_epoch, start_epoch + n_epochs):
    print ("\nepoch = "+str(epoch))
    loss_train = train(LSTM_en, linear_model, DataLoader = nli_train_loader,
                       criterion = nn.NLLLoss(),
                       optimizer = torch.optim.Adam(list(LSTM_en.parameters()) + list(linear_model.parameters()), 
                                                   lr=config.lr, weight_decay=0),
                       epoch = epoch)
    val_acc = accuracy(LSTM_en, linear_model, nli_dev_loader, nn.NLLLoss(reduction='sum'))
    print ("\n{} Validation Accuracy = {}".format(config.val_test_lang.upper(), val_acc))
    if val_acc <= validation_accuracy[-1]:
        break
    validation_accuracy.append(val_acc)
    torch.save(LSTM_en.state_dict(), "best_encoder_eng_mnli_{}_{}".format(epoch, config.experiment_lang))
    torch.save(linear_model.state_dict(), "best_linear_eng_mnli_{}_{}".format(epoch, config.experiment_lang))
    

Encoder:
 biLSTM(
  (embedding): Embedding(400002, 300)
  (drop_out): Dropout(p=0.09)
  (LSTM): LSTM(300, 512, batch_first=True, bidirectional=True)
  (bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)
Classifier:
 Linear_Layers(
  (mlp): Sequential(
    (0): Linear(in_features=4096, out_features=128, bias=True)
    (1): ReLU(inplace)
    (2): Dropout(p=0.1)
    (3): Linear(in_features=128, out_features=3, bias=True)
  )
)

epoch = 0
Train Epoch: 0 [392320/392702 (100%)]	Loss: 1.009115
EN Validation Accuracy = 64.49799537658691

epoch = 1
Train Epoch: 1 [392320/392702 (100%)]	Loss: 0.906441
EN Validation Accuracy = 65.66265225410461

epoch = 2
Train Epoch: 2 [392320/392702 (100%)]	Loss: 0.895451
EN Validation Accuracy = 66.38554334640503

epoch = 3
Train Epoch: 3 [392320/392702 (100%)]	Loss: 0.795442
EN Validation Accuracy = 68.07228922843933

epoch = 4
Train Epoch: 4 [392320/392702 (100%)]	Loss: 0.969998
EN Validation Accuracy = 67.2690749168396
